<a href="https://colab.research.google.com/github/M-Awwab-Khan/decision-tree-classifier-implementation-python/blob/main/Decision_Tree_Implementation_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt